# Chapter 1: ROS 2 Fundamentals

This notebook introduces the fundamental concepts of ROS 2 (Robot Operating System 2), including nodes, topics, services, and actions. We'll explore these concepts both in simulation and with hardware fallbacks.

In [ ]:
# Configuration cell - select execution mode
EXECUTION_MODE = "simulation"  # Options: "hardware", "simulation"

print(f"ROS 2 Chapter 1: Running in {EXECUTION_MODE} mode")
print("Initializing ROS 2 environment...")

# Import required libraries
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

# Check if running in simulation mode
if EXECUTION_MODE == "simulation":
    print("Using simulation mode - no physical hardware required")
    # Mock ROS 2 classes for simulation
    class MockNode:
        def __init__(self, name):
            self.name = name
            print(f"MockNode '{name}' initialized")
        
        def create_publisher(self, msg_type, topic_name, qos_profile=None):
            print(f"Mock publisher created for topic: {topic_name}")
            return MockPublisher(topic_name)
        
        def create_subscription(self, msg_type, topic_name, callback, qos_profile=None):
            print(f"Mock subscription created for topic: {topic_name}")
            return MockSubscription(topic_name, callback)
    
    class MockPublisher:
        def __init__(self, topic_name):
            self.topic_name = topic_name
        
        def publish(self, msg):
            print(f"Publishing to {self.topic_name}: {msg}")
    
    class MockSubscription:
        def __init__(self, topic_name, callback):
            self.topic_name = topic_name
            self.callback = callback
    
    # Mock ROS 2 initialization
    def init():
        print("Mock ROS 2 initialized")
    
    def spin_once(node, timeout_sec=None):
        print("Mock spin_once called")
        time.sleep(0.1)  # Simulate processing time
    
    def shutdown():
        print("Mock ROS 2 shutdown")
    
    # Assign mock functions to rclpy equivalent names
    rclpy = type('MockRclpy', (), {
        'init': init,
        'spin_once': spin_once,
        'shutdown': shutdown
    })()
    
    Node = MockNode
else:
    print("Using hardware mode - connecting to physical ROS 2 system")
    import rclpy
    from rclpy.node import Node
    from std_msgs.msg import String

print("Environment setup complete!")

## 1. Creating a ROS 2 Node

In ROS 2, a node is the fundamental unit of execution. Nodes are processes that perform computation. Let's create a simple node that publishes messages.

In [ ]:
# Create a simple publisher node
class TalkerNode(Node):
    def __init__(self):
        super().__init__('talker')
        self.publisher = self.create_publisher(String, 'chatter', 10)
        self.counter = 0
        self.timer = self.create_timer(0.5, self.timer_callback)  # Publish every 0.5 seconds
        self.get_logger().info('Talker node initialized')
    
    def timer_callback(self):
        msg = String()
        msg.data = f'Hello ROS 2! Count: {self.counter}'
        self.publisher.publish(msg)
        self.get_logger().info(f'Publishing: {msg.data}')
        self.counter += 1

# Initialize ROS 2 and create the node
rclpy.init()
talker = TalkerNode()

print("Talker node created successfully")

## 2. Creating a Subscriber Node

Now let's create a subscriber node that listens to the messages published by our talker node.

In [ ]:
# Create a simple subscriber node
class ListenerNode(Node):
    def __init__(self):
        super().__init__('listener')
        self.subscription = self.create_subscription(
            String,
            'chatter',
            self.listener_callback,
            10)
        self.subscription  # Prevent unused variable warning
        self.get_logger().info('Listener node initialized')
    
    def listener_callback(self, msg):
        self.get_logger().info(f'I heard: {msg.data}')

# Create the listener node
listener = ListenerNode()
print("Listener node created successfully")

## 3. Running the Nodes

In a real ROS 2 system, we would run both nodes simultaneously. For this simulation, we'll demonstrate the concept by simulating the interaction.

In [ ]:
# Simulate running both nodes for a short time
if EXECUTION_MODE == "simulation":
    print("\n--- Simulating ROS 2 communication ---")
    for i in range(5):  # Simulate 5 message exchanges
        # Simulate talker publishing
        msg_content = f'Hello ROS 2! Count: {i}'
        print(f'Talker: Publishing -> {msg_content}')
        
        # Simulate listener receiving
        print(f'Listener: Received <- {msg_content}')
        time.sleep(0.5)
    
    print("\nSimulation complete!")
else:
    print("\nIn hardware mode, you would run both nodes simultaneously")
    print("For now, we'll just verify the nodes are created correctly")
    
    # In a real system, we would use multi-threading or separate processes
    # For this notebook, we'll just verify the nodes exist
    assert talker is not None, "Talker node should exist"
    assert listener is not None, "Listener node should exist"
    print("Node verification successful!")

## 4. Services in ROS 2

Services provide a request/reply communication pattern. Let's create a simple service example.

In [ ]:
# For simulation mode, create mock service classes
if EXECUTION_MODE == "simulation":
    class MockService:
        def __init__(self, service_name, callback):
            self.service_name = service_name
            self.callback = callback
            print(f"Mock service '{service_name}' created")
        
        def send_response(self, response, request_header):
            print(f"Sending response: {response}")
    
    def create_service(node, srv_type, srv_name, callback):
        print(f"Creating service: {srv_name}")
        return MockService(srv_name, callback)
    
    def create_client(node, srv_type, srv_name):
        print(f"Creating client for service: {srv_name}")
        return MockService(srv_name, lambda req, response: response)
else:
    from example_interfaces.srv import AddTwoInts
    
    def create_service(node, srv_type, srv_name, callback):
        return node.create_service(srv_type, srv_name, callback)
    
    def create_client(node, srv_type, srv_name):
        client = node.create_client(srv_type, srv_name)
        while not client.wait_for_service(timeout_sec=1.0):
            print('Service not available, waiting again...')
        return client

# Example service callback
def add_two_ints_callback(request, response):
    response.sum = request.a + request.b
    print(f'Request: {request.a} + {request.b} = {response.sum}')
    return response

print("Service setup complete")

## 5. Visualization

Let's visualize the communication pattern between nodes.

In [ ]:
# Create a simple visualization of the ROS 2 communication
fig, ax = plt.subplots(figsize=(10, 6))

# Simulate message timing
times = np.arange(0, 5, 0.5)
talker_messages = [f'Hello ROS 2! Count: {i}' for i in range(len(times))]

# Plot message exchange
ax.scatter(times, [1]*len(times), c='blue', label='Talker Publishes', s=100, zorder=5)
ax.scatter(times + 0.1, [0]*len(times), c='red', label='Listener Receives', s=100, zorder=5)

ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Node')
ax.set_title('ROS 2 Message Exchange Simulation')
ax.set_yticks([0, 1])
ax.set_yticklabels(['Listener', 'Talker'])
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Visualization complete - showing message timing between nodes")

## Summary

In this notebook, we've covered:
1. Basic ROS 2 concepts: nodes, publishers, subscribers
2. How to create publisher and subscriber nodes
3. The service communication pattern
4. Visualization of message exchange

These concepts form the foundation for more complex ROS 2 applications. In the next chapter, we'll explore simulation environments where we can test these concepts without requiring physical hardware.

In [ ]:
# Clean up
if EXECUTION_MODE == "simulation":
    print("\nCleaning up simulation resources...")
else:
    # In hardware mode, we would properly shut down the nodes
    print("\nShutting down ROS 2 nodes...")
    # talker.destroy_node()
    # listener.destroy_node()
    # rclpy.shutdown()

print("Chapter 1 complete! You've learned the fundamentals of ROS 2.")